<H1 style="color:blue";> Introduction</H1>

<H2 style="color:blue";> BackGround </H2>
<H4 style="color:blue";> Assuming a client is asking for advices of openning a restaurant in Toronto,</H4> 
<H4 style="color:blue";> 1.where to open it ?</H4>
<H4 style="color:blue";> 2.what type of restaurant to open ?</H4>  
<H4 style="color:blue";>how do we find a proper answer for the client ? </H4>

<H1 style="color:blue";> Description of the data </H1>
<H4 style="color:blue";>To answer that, we need collect some data, a Toronto dataset include:</H4>
<H4 style="color:blue";> 1. "Postal Code" ,"Borough", "Neighborhood", Location("Lat", "Lng") of current restaurants</H4>
<H4 style="color:blue";> 2. then analyze these data to find out what type of restaurants are there and how are they distributed in Toronto City</H4>
<H4 style="color:blue";> Put these together, we can get some clue to decide where to open what type of restaurant</H4>

<H1 style="color:blue";> Methodology </H1>
<H4 style="color:blue";> I will use 4square API to find out restaurant information and utilize the coursera module's Toronto neighbourhood data</H4>
<H4 style="color:blue";>We can understand more clearly and straighforward after visualizing and tablizing our collected data</H4>

<H1 style="color:green";> Collect restaurant distribution information</H1>

### Import required Module

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

## <font color=#008000>1. Find Borough, Neighborhood </font> 

### Configure wiki url of Toronto and download the PostalCode data

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

### Use bs4 to parsing the the download Data

In [3]:
NeighborhoodsinToronto=BeautifulSoup(html_data,"html.parser")

In [4]:
#print(NeighborhoodsinToronto.prettify())

### Next use split ,strip and replace functions for getting Borough and Neighborhood information.

In [5]:
table_contents=[]
table=NeighborhoodsinToronto.find('tbody')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [6]:
# print(table_contents)
df_NeighborhoodsinToronto=pd.DataFrame(table_contents)
df_NeighborhoodsinToronto['Borough']=df_NeighborhoodsinToronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Print The Result (postal code, borough, neighborhood)

In [7]:
df_NeighborhoodsinToronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df_NeighborhoodsinToronto.shape

(103, 3)

## <font color=#008000>2. Find Latitude, Longitude </font> 

### Read "Geospatial_Coordinates.csv" into dataframe

In [9]:
Geo_coor=pd.read_csv('Geospatial_Coordinates.csv') 

In [10]:
Geo_coor

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## Combine Neighborhood information and coordination into one frame

In [11]:
df_Neighborhood_latlng = pd.merge(df_NeighborhoodsinToronto, 
                     Geo_coor, 
                     on ='Postal Code', 
                     how ='left')
df_Neighborhood_latlng

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
import numpy as np # library to handle data in a vectorized manner

# import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Creating Toronto Map

In [13]:
# create map of Toronto using latitude and longitude values
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_Neighborhood_latlng['Latitude'], df_Neighborhood_latlng['Longitude'], df_Neighborhood_latlng['Borough'], df_Neighborhood_latlng['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## <font color=#008000> 3. Find Restaurant </font> 

## Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'TR2SUDHKHBJRAOHIP3AYUGZOURV3LTTXTR4UJ0QC2TPZUMQY' # your Foursquare ID
CLIENT_SECRET = 'CTRH4KCENV4TGOI1AWNEPV1BB1JH00MHBPVH3VTFT13DGIQR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TR2SUDHKHBJRAOHIP3AYUGZOURV3LTTXTR4UJ0QC2TPZUMQY
CLIENT_SECRET:CTRH4KCENV4TGOI1AWNEPV1BB1JH00MHBPVH3VTFT13DGIQR


### From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## <font color=#090000> Explore Toronto for all Retaurant - start

In [17]:
def getNearbyVenues1(names, latitudes, longitudes, radius=500):
    
    venues_list1=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results1 = requests.get(url1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list1.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results1])

    nearby_venues = pd.DataFrame([item for venue_list1 in venues_list1 for item in venue_list1])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
all_toronto_data = df_Neighborhood_latlng.reset_index(drop=True)
all_toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [19]:
# 
all_toronto_venues = getNearbyVenues1(
                                      names=all_toronto_data['Neighborhood'],
                                      latitudes=all_toronto_data['Latitude'],
                                      longitudes=all_toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [20]:
# Let's check the size of the resulting dataframe
print(all_toronto_venues.shape)
all_toronto_venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [21]:
all_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,60,60,60,60,60,60
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16


In [22]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(all_toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


In [23]:
all_toronto_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,2,2,2,2,2,2
Adult Boutique,1,1,1,1,1,1
Airport,2,2,2,2,2,2
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,1,1,1,1,1,1
Airport Service,2,2,2,2,2,2
Airport Terminal,2,2,2,2,2,2
American Restaurant,22,22,22,22,22,22


### Combine "Postal Code","Borough","Neighborhood","Lat","Lng","Venue","Venue Category" info together

In [24]:
# Combine "Postal Code","Borough","Neighborhood","Lat","Lng","Venue","Venue Category" info together
all_toronto_venues_postalcode = pd.merge(all_toronto_data[['Postal Code','Borough','Neighborhood']],all_toronto_venues, 
                     on ='Neighborhood', 
                     how ='left')
all_toronto_venues_postalcode.head()

,Postal Code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [25]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(all_toronto_venues_postalcode['Venue Category'].unique())))

There are 276 uniques categories.


<H2 style="color:green";> Filter food categories only </H2>

In [26]:
# TEST CODE
#list(set(all_toronto_venues_postalcode['Venue Category']))
#somevalue=['Restaurant']
#all_toronto_venues_postalcode_food=all_toronto_venues_postalcode.loc[all_toronto_venues_postalcode['Venue Category'].isin(somevalue)]
#all_toronto_venues_postalcode_food
# ~ TEST CODE

In [27]:
str_choice = "Restaurant|Coffee|Drink|Sandwich|Beer|Food|Diner" 
all_toronto_venues_postalcode_food = all_toronto_venues_postalcode[all_toronto_venues_postalcode['Venue Category'].str.contains(str_choice, na=False)]
print(all_toronto_venues_postalcode_food.shape)
all_toronto_venues_postalcode_food.head()

(769, 9)


,Postal Code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,M3A,North York,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,M4A,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
9,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


## Explore Toronto for all Retaurants - End

<H1 style="color:Blue";> Analyze Restaurants in Each Neighborhood  </H1>


In [28]:
all_toronto_venues_postalcode_food.shape

(769, 9)

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(all_toronto_venues_postalcode_food[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = all_toronto_venues_postalcode_food['Postal Code']
toronto_onehot['Neighborhood'] = all_toronto_venues_postalcode_food['Neighborhood'] 
toronto_onehot['Venue'] = all_toronto_venues_postalcode_food['Venue']

# move Venue column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# move Neighborhood column to the first column (step2)
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# move Postal Code column to the first column (step3)
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

#================================


(769, 63)


,Postal Code,Neighborhood,Venue,Airport Food Court,American Restaurant,Asian Restaurant,Beer Bar,Beer Store,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Health Food Store,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean BBQ Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
1,M3A,Parkwoods,KFC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M3A,Parkwoods,Variety Store,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4A,Victoria Village,Portugril,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,M4A,Victoria Village,Tim Hortons,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,M5A,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
#toronto_onehot.to_csv('toronoto_onehot.csv')

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [31]:
all_toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(all_toronto_grouped.shape)
all_toronto_grouped.head()

(75, 61)


,Neighborhood,Airport Food Court,American Restaurant,Asian Restaurant,Beer Bar,Beer Store,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Health Food Store,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean BBQ Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.0,0.0,0.142857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.066667,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.133333,0.0,0.0,0.066667,0.0,0.066667,0.0,0.0,0.0,0.0,0.0


In [32]:
#all_toronto_grouped.to_csv('all_toronto_grouped.csv')

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in all_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = all_toronto_grouped[all_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print(temp)
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   1.0
1         Airport Food Court   0.0
2        American Restaurant   0.0
3          Health Food Store   0.0
4          Indian Restaurant   0.0


----Alderwood, Long Branch----
                     venue  freq
0              Coffee Shop   0.5
1           Sandwich Place   0.5
2       Airport Food Court   0.0
3  New American Restaurant   0.0
4        Indian Restaurant   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.29
1                      Diner  0.14
2  Middle Eastern Restaurant  0.14
3                 Restaurant  0.14
4             Sandwich Place  0.14


----Bayview Village----
                     venue  freq
0      Japanese Restaurant   0.5
1       Chinese Restaurant   0.5
2       Airport Food Court   0.0
3  New American Restaurant   0.0
4        Indian Restaurant   0.0


----Bedford Park, Lawrence Manor East----
     

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_toronto_grouped['Neighborhood']

for ind in np.arange(all_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(75, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Vietnamese Restaurant,German Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
1,"Alderwood, Long Branch",Sandwich Place,Coffee Shop,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Sandwich Place,Diner,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
3,Bayview Village,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Thai Restaurant,Greek Restaurant,Sushi Restaurant,Fast Food Restaurant


<H2 style="color:green";> Cluster Neighborhoods </H2>


Run _k_-means to cluster the neighborhood into 5 clusters.


In [36]:
# set number of clusters
kclusters = 5

all_toronto_grouped_clustering = all_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:103] 

array([4, 1, 4, 4, 4, 4, 1, 1, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 4, 4, 4, 4,
       2, 1, 3, 4, 1, 4, 4, 3, 4, 4, 3, 1, 4, 0, 0, 1, 4, 4, 1, 4, 4, 0,
       0, 3, 3, 4, 1, 1, 4, 0, 1, 4, 4, 0, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4,
       4, 1, 2, 4, 2, 4, 1, 1, 4])

In [37]:
print(all_toronto_grouped.shape)
all_toronto_grouped.head()

(75, 61)


,Neighborhood,Airport Food Court,American Restaurant,Asian Restaurant,Beer Bar,Beer Store,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Health Food Store,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean BBQ Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.0,0.0,0.142857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.066667,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.133333,0.0,0.0,0.066667,0.0,0.066667,0.0,0.0,0.0,0.0,0.0


In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(75, 12)


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Agincourt,Latin American Restaurant,Vietnamese Restaurant,German Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
1,1,"Alderwood, Long Branch",Sandwich Place,Coffee Shop,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Sandwich Place,Diner,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
3,4,Bayview Village,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
4,4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Thai Restaurant,Greek Restaurant,Sushi Restaurant,Fast Food Restaurant


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [39]:
# Merge all data(Postal Code, Borough, Lat, Lng, Cluster Labels, Common Venue ranking)

all_toronto_merged = all_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
all_toronto_merged = all_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(all_toronto_merged.shape)
all_toronto_merged.head() # check the last columns!

(103, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Fast Food Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Food Truck,Food Court,Food,Filipino Restaurant,Falafel Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,French Restaurant,Beer Store,Restaurant,Mexican Restaurant,Asian Restaurant,Food Court,Food & Drink Shop,Food,Doner Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Vietnamese Restaurant,Coffee Shop,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Sandwich Place,Diner,Mexican Restaurant,Beer Bar,Fast Food Restaurant,Falafel Restaurant


In [40]:
# DropNaN

all_toronto_merged.dropna(axis=0, inplace=True)
print(all_toronto_merged.shape)
print("\n")
print(all_toronto_merged[all_toronto_merged['Cluster Labels'].isnull()])

(75, 16)


Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood, Latitude, Longitude, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []


## Finally, let's visualize the resulting clusters


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_toronto_merged['Latitude'], all_toronto_merged['Longitude'], all_toronto_merged['Neighborhood'], all_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(cluster-1)],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters


#### Cluster 1


In [42]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 0, all_toronto_merged.columns[[2] + list(range(5, all_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0.0,Food & Drink Shop,Fast Food Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Food Truck,Food Court,Food,Filipino Restaurant,Falafel Restaurant
6,"Malvern, Rouge",0.0,Fast Food Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Falafel Restaurant
27,Hillcrest Village,0.0,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,German Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Falafel Restaurant
47,"India Bazaar, The Beaches West",0.0,Fast Food Restaurant,Italian Restaurant,Sandwich Place,Food & Drink Shop,Sushi Restaurant,Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.0,Fast Food Restaurant,Turkish Restaurant,Caribbean Restaurant,Sandwich Place,Falafel Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Vietnamese Restaurant
82,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,Fast Food Restaurant,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food
89,"South Steeles, Silverstone, Humbergate, Jamest...",0.0,Beer Store,Fast Food Restaurant,Sandwich Place,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant
90,"Steeles West, L'Amoreaux West",0.0,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food
102,"Mimico NW, The Queensway West, South of Bloor,...",0.0,Fast Food Restaurant,Sandwich Place,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Falafel Restaurant


#### Cluster 2


In [43]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 1, all_toronto_merged.columns[[2] + list(range(5, all_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,1.0,Coffee Shop,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
2,"Regent Park, Harbourfront",1.0,Coffee Shop,French Restaurant,Beer Store,Restaurant,Mexican Restaurant,Asian Restaurant,Food Court,Food & Drink Shop,Food,Doner Restaurant
3,"Lawrence Manor, Lawrence Heights",1.0,Vietnamese Restaurant,Coffee Shop,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
4,Ontario Provincial Government,1.0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Sandwich Place,Diner,Mexican Restaurant,Beer Bar,Fast Food Restaurant,Falafel Restaurant
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",1.0,Beer Store,Coffee Shop,Vietnamese Restaurant,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant
22,Woburn,1.0,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
34,"Northwood Park, York University",1.0,Caribbean Restaurant,Coffee Shop,Vietnamese Restaurant,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant
36,"Harbourfront East, Union Station, Toronto Islands",1.0,Coffee Shop,Restaurant,Italian Restaurant,New American Restaurant,Japanese Restaurant,Mexican Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Sushi Restaurant,Indian Restaurant
38,"Kennedy Park, Ionview, East Birchmount Park",1.0,Coffee Shop,Vietnamese Restaurant,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
43,"Brockton, Parkdale Village, Exhibition Place",1.0,Coffee Shop,Italian Restaurant,Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant


#### Cluster 3


In [44]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 2, all_toronto_merged.columns[[2] + list(range(5, all_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"West Deane Park, Princess Gardens, Martin Grov...",2.0,Middle Eastern Restaurant,Vietnamese Restaurant,German Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
31,"Dufferin, Dovercourt Village",2.0,Brazilian Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
71,"Wexford, Maryvale",2.0,Middle Eastern Restaurant,Sandwich Place,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant


#### Cluster 4


In [45]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 3, all_toronto_merged.columns[[2] + list(range(5, all_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,"Guildwood, Morningside, West Hill",3.0,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
51,"Cliffside, Cliffcrest, Scarborough Village West",3.0,American Restaurant,Vietnamese Restaurant,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
68,Forest Hill North & West,3.0,Sushi Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
83,"Moore Park, Summerhill East",3.0,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
88,"New Toronto, Mimico South, Humber Bay Shores",3.0,Mexican Restaurant,American Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Seafood Restaurant,Ethiopian Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
100,Enclave of M4L,3.0,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Falafel Restaurant


#### Cluster 5


In [46]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 4, all_toronto_merged.columns[[2] + list(range(5, all_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Don Mills North,4.0,Japanese Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
9,"Garden District, Ryerson",4.0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Fast Food Restaurant,Diner,Vietnamese Restaurant,Ethiopian Restaurant,Mexican Restaurant
10,Glencairn,4.0,Italian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Diner,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
13,Don Mills South,4.0,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Dim Sum Restaurant,Asian Restaurant,Beer Store,Sushi Restaurant,Chinese Restaurant,Eastern European Restaurant
14,Woodbine Heights,4.0,Beer Store,Vietnamese Restaurant,Diner,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant
15,St. James Town,4.0,Coffee Shop,Italian Restaurant,Restaurant,American Restaurant,Moroccan Restaurant,Seafood Restaurant,Diner,Comfort Food Restaurant,Middle Eastern Restaurant,New American Restaurant
19,The Beaches,4.0,Health Food Store,Vietnamese Restaurant,German Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
20,Berczy Park,4.0,Coffee Shop,Beer Bar,Seafood Restaurant,Restaurant,Diner,French Restaurant,Eastern European Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Greek Restaurant
23,Leaside,4.0,Coffee Shop,Middle Eastern Restaurant,Beer Store,Sushi Restaurant,Mexican Restaurant,Sandwich Place,Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Diner
24,Central Bay Street,4.0,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Korean Restaurant,Modern European Restaurant,Indian Restaurant,Diner,Portuguese Restaurant,Restaurant
